In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import datetime

pd.set_option('display.max_columns', 150)

### Get Teams

In [35]:
# dfTeams = pd.read_csv('teams.csv')
# dfTeams.conference = dfTeams.conference.apply(lambda c: c.split(' (')[0])

In [43]:
dfTeams.head()

,year,tm_link,conference,team
0,2001,duke,ACC,Duke
1,2001,arizona,Pac-10,Arizona
2,2001,stanford,Pac-10,Stanford
3,2001,michigan-state,Big Ten,Michigan State
4,2001,maryland,ACC,Maryland


### Get Game Summaries

In [44]:
dfGames = pd.read_csv('games_summary.csv')

/home/chris/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1159: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [59]:
dfGames['dt'] = dfGames.box.apply(lambda g: datetime.datetime.strptime(g[:10], "%Y-%m-%d").date() )
dfGames['season'] = dfGames.dt.apply(lambda d:  d.year+1 if d.month > 4 else d.year)

REORDER_GAME_COLUMNS = ['season', 'Team', 'G', 'Date', 'Year', 'dt', 'box', 'Type', 'Opponent', 'Conf', 'Result', 'Tm', 'Opp', 'OT', 'W', 'L', 'Streak', 'Network', 'Time']
dfGames = dfGames[REORDER_GAME_COLUMNS]

dfGames = dfGames.merge(dfTeams[['year', 'team', 'tm_link']],
                        how = 'left',
                        left_on = ['season', 'Opponent'],
                        right_on = ['year', 'team'],
                        suffixes = ['', '_y']
                        )
dfGames.rename(columns = {'tm_link' : 'opp_link'}, inplace = True) 
dfGames.drop(['Network', 'Time', 'Date', 'year', 'team'], inplace = True, axis = 1)

dfGames.columns = [c.lower() for c in dfGames.columns]

In [79]:
dfGames.head()

,season,team,g,year,dt,box,type,opponent,conf,result,tm,opp,ot,w,l,streak,opp_link
0,2011,ohio-state,1,2011,2010-11-12,2010-11-12-ohio-state,REG,North Carolina A&T,MEAC,W,102,61,NaN,1,0,W 1,north-carolina-at
1,2011,ohio-state,2,2011,2010-11-16,2010-11-16-florida,REG,Florida,SEC,W,93,75,NaN,2,0,W 2,florida
2,2011,ohio-state,3,2011,2010-11-20,2010-11-20-ohio-state,REG,North Carolina-Wilmington,CAA,W,81,41,NaN,3,0,W 3,north-carolina-wilmington
3,2011,ohio-state,4,2011,2010-11-23,2010-11-23-ohio-state,REG,Morehead State,OVC,W,64,45,NaN,4,0,W 4,morehead-state
4,2011,ohio-state,5,2011,2010-11-26,2010-11-26-ohio-state,REG,Miami (OH),MAC,W,66,45,NaN,5,0,W 5,miami-oh


### Get TeamGame Stats

In [85]:
df_stats = pd.read_csv('player_stats.csv')
df_stats.drop(df_stats.columns[0], axis = 1, inplace= True)
df_stats_backup = df_stats.copy()


REORDER_COLS = ['player_id', 'player', 'team', 'gm', 'mp', 'fg', 'fga', 'fg_pct', 'fg2', 'fg2a', 'fg2_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']

df_stats = df_stats[REORDER_COLS]
df_stats.insert(3, 'dt', df_stats.gm.apply(lambda g: datetime.datetime.strptime(g[:10], "%Y-%m-%d").date() ))
df_stats.sort(['dt', 'gm', 'team', 'player_id'], inplace = True)

df_stats.reset_index(inplace = True, drop = True)

df_stats.insert(4, 'season', df_stats.dt.apply(lambda d:  d.year+1 if d.month > 4 else d.year))
df_stats.insert(6, 'home_team', df_stats.apply(lambda r: r.team in r.gm, axis = 1))

In [86]:
numb_cols = df_stats.columns[7:]

In [87]:
dfTeamGames = df_stats.groupby(['season', 'dt', 'gm', 'team', 'home_team'], as_index = False)[numb_cols].sum()

In [93]:
dfTeamGames = dfTeamGames.merge(dfGames[['season', 'team', 'opp_link', 'box']],
                                how = 'left',
                                left_on = ['season', 'team', 'gm'],
                                right_on = ['season', 'team', 'box'],
                                suffixes = ['', '_y']
                                )

dfTeamGames.drop('box', axis = 1, inplace = True)

In [95]:
dfTeamGames = dfTeamGames.merge(dfTeams[['year', 'tm_link', 'conference']],
                                how = 'left',
                                left_on = ['season', 'team'],
                                right_on = ['year', 'tm_link'],
                                suffixes = ['', '_y']
                                )

dfTeamGames.drop(['year', 'tm_link'], axis = 1, inplace = True)

In [98]:
dfTeamGames = dfTeamGames.merge(dfTeams[['year', 'tm_link', 'conference']],
                                how = 'left',
                                left_on = ['season', 'opp_link'],
                                right_on = ['year', 'tm_link'],
                                suffixes = ['', '_y']
                                )

dfTeamGames.drop(['year', 'tm_link'], axis = 1, inplace = True)
dfTeamGames.rename(columns = {'conference_y' : 'opp_conference'}, inplace = True)

In [103]:
dfTeamGames.insert(5,'conf_game',  dfTeamGames.conference == dfTeamGames.opp_conference)

In [ ]:
for c in dfTeamGames.columns:
    if 'pct' in c:
        print c.split('_')[0], c.split('_')[0]+'a'
        dfTeamGames[c] = dfTeamGames[c.split('_')[0]].map(float)/dfTeamGames[c.split('_')[0]+'a'].map(float)

In [119]:
dfTeamGames = dfTeamGames.merge(dfTeamGames[['gm', 'team'] + numb_cols.tolist()],
                                how = 'left',
                                left_on = ['gm', 'opp_link'],
                                right_on = ['gm', 'team'],
                                suffixes = ['', '_opp']
                                )

In [125]:
dfTeamGames.insert(6, 'result_win', dfTeamGames.pts > dfTeamGames.pts_opp)

In [134]:
for nc in numb_cols[1:]:
    colName = nc + '_net'
    dfTeamGames[colName] = dfTeamGames[nc] - dfTeamGames[nc + '_opp']

In [137]:
dfTeamGames.tail(6)

,season,dt,gm,team,home_team,conf_game,result_win,mp,fg,fga,fg_pct,fg2,fg2a,fg2_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts,opp_link,conference,opp_conference,team_opp,mp_opp,fg_opp,fga_opp,fg_pct_opp,fg2_opp,fg2a_opp,fg2_pct_opp,fg3_opp,fg3a_opp,fg3_pct_opp,ft_opp,fta_opp,ft_pct_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,fg_net,fga_net,fg_pct_net,fg2_net,fg2a_net,fg2_pct_net,fg3_net,fg3a_net,fg3_pct_net,ft_net,fta_net,ft_pct_net,orb_net,drb_net,trb_net,ast_net,stl_net,blk_net,tov_net,pf_net,pts_net
58388,2015,2015-04-04,2015-04-04-duke,duke,True,False,True,200,26,50,0.520000,24,40,0.600000,2,10,0.200000,27,37,0.729730,7,29,36,10,9,4,8,18,81,michigan-state,ACC,Big Ten,michigan-state,200,22,55,0.400000,15,35,0.428571,7,20,0.350000,10,16,0.625000,8,23,31,10,4,3,14,28,61,4,-5,0.120000,9,5,0.171429,-5,-10,-0.150000,17,21,0.104730,-1,6,5,0,5,1,-6,-10,20
58389,2015,2015-04-04,2015-04-04-duke,michigan-state,False,False,False,200,22,55,0.400000,15,35,0.428571,7,20,0.350000,10,16,0.625000,8,23,31,10,4,3,14,28,61,duke,Big Ten,ACC,duke,200,26,50,0.520000,24,40,0.600000,2,10,0.200000,27,37,0.729730,7,29,36,10,9,4,8,18,81,-4,5,-0.120000,-9,-5,-0.171429,5,10,0.150000,-17,-21,-0.104730,1,-6,-5,0,-5,-1,6,10,-20
58390,2015,2015-04-04,2015-04-04-kentucky,kentucky,True,False,False,200,26,54,0.481481,23,49,0.469388,3,5,0.600000,9,10,0.900000,6,16,22,7,5,4,6,20,64,wisconsin,SEC,Big Ten,wisconsin,200,23,48,0.479167,16,31,0.516129,7,17,0.411765,18,22,0.818182,9,21,30,10,1,2,10,14,71,3,6,0.002315,7,18,-0.046741,-4,-12,0.188235,-9,-12,0.081818,-3,-5,-8,-3,4,2,-4,6,-7
58391,2015,2015-04-04,2015-04-04-kentucky,wisconsin,False,False,True,200,23,48,0.479167,16,31,0.516129,7,17,0.411765,18,22,0.818182,9,21,30,10,1,2,10,14,71,kentucky,Big Ten,SEC,kentucky,200,26,54,0.481481,23,49,0.469388,3,5,0.600000,9,10,0.900000,6,16,22,7,5,4,6,20,64,-3,-6,-0.002315,-7,-18,0.046741,4,12,-0.188235,9,12,-0.081818,3,5,8,3,-4,-2,4,-6,7
58392,2015,2015-04-06,2015-04-06-duke,duke,True,False,True,200,24,51,0.470588,20,40,0.500000,4,11,0.363636,16,20,0.800000,6,27,33,7,4,6,5,13,68,wisconsin,ACC,Big Ten,wisconsin,200,25,61,0.409836,18,40,0.450000,7,21,0.333333,6,10,0.600000,9,24,33,13,3,1,5,15,63,-1,-10,0.060752,2,0,0.050000,-3,-10,0.030303,10,10,0.200000,-3,3,0,-6,1,5,0,-2,5
58393,2015,2015-04-06,2015-04-06-duke,wisconsin,False,False,False,200,25,61,0.409836,18,40,0.450000,7,21,0.333333,6,10,0.600000,9,24,33,13,3,1,5,15,63,duke,Big Ten,ACC,duke,200,24,51,0.470588,20,40,0.500000,4,11,0.363636,16,20,0.800000,6,27,33,7,4,6,5,13,68,1,10,-0.060752,-2,0,-0.050000,3,10,-0.030303,-10,-10,-0.200000,3,-3,0,6,-1,-5,0,2,-5


In [138]:
# dfTeamGames.to_csv('team_level_game_stats.csv', index = False)

##### Scale Game Stats

In [160]:
all_num_cols = list(numb_cols[1:])
all_num_cols += numb_cols[1:]+'_opp'
all_num_cols += numb_cols[1:]+'_net'

dfTG_scaled = dfTeamGames.copy()
dfTG_scaled.loc[:, all_num_cols] = dfTG_scaled.loc[:, all_num_cols].apply(lambda c: (c - c.mean())/c.std(), axis = 0)
dfTG_scaled[c] = (dfTG_scaled[c] - dfTG_scaled[c].mean())/dfTG_scaled[c].std()

In [195]:
# dfTG_scaled[['pts']].dropna().corrwith(dfTG_scaled[[c for c in all_num_cols if 'pct' not in c]].dropna())
corr = dfTG_scaled[numb_cols].corrwith(dfTG_scaled.result_win)
print corr.sort(ascending = False)
# dfTG_scaled[all_num_cols].corr().to_csv('stats_corr_matrix.csv')

None


In [170]:
dfTG_scaled.describe().T

,count,mean,std,min,25%,50%,75%,max
season,58394,2013.019,1.414479,2011,2012,2013,2014,2015
home_team,58394,0.5022091,0.4999994,False,0,1,1,True
conf_game,58394,0.5513923,0.4973561,False,0,1,1,True
result_win,58394,0.4663493,0.4988706,False,0,0,1,True
mp,58394,201.7821,7.830139,102,200,200,200,325
fg,58394,3.166134e-16,1,-4.165575,-0.7763096,0.0211646,0.6192702,6.400958
fga,58394,-3.757502e-16,1,-3.691883,-0.6664683,-0.008769468,0.6489293,6.568219
fg_pct,58394,9.661454e-17,1,-4.738463,-0.692578,-0.01992344,0.694725,4.379437
fg2,58394,-2.392244e-16,1,-3.25297,-0.754094,-0.129375,0.7035835,6.950773
fg2a,58394,4.297765e-16,1,-3.678205,-0.6297202,-0.09954897,0.6957079,5.334706
